In [8]:
import os
import numpy as np
import scanpy as sc
import pandas as pd
import glob


%matplotlib widget
path = "/home/olle/PycharmProjects/LODE/workspace/feature_statistics/cell_data"

cell_pd = pd.read_csv(os.path.join(path, "feature_statistics.csv"))

In [3]:
# create adata object
var_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15"]

X = np.array(cell_pd[var_names])

obs_id = cell_pd.id
obs_cls = cell_pd.id.str.split("-", expand=True)[0]


adata = sc.AnnData(X=X)
adata.obs["obs_id"] = obs_id.values.tolist()
adata.obs["obs_cls"] = obs_cls.values.tolist()
adata.var["var_name"] = var_names

In [ ]:
#### Add train test split to obs

In [4]:
test_image_path = "/home/olle/PycharmProjects/LODE/workspace/feature_statistics/cell_data/OCT2017/test"

test_ids_list = [i.split("/")[-1] for i in glob.glob(test_image_path + "/*/*")]

all_ids = pd.DataFrame(adata.obs["obs_id"])
test_ids = pd.DataFrame(test_ids_list)
test_paths = pd.DataFrame([test_ids_list, glob.glob(test_image_path + "/*/*")]).T

data_split_pd = pd.merge(all_ids, test_ids, left_on="obs_id", right_on=0, how="left")

data_split_pd["split"] = "train"
data_split_pd["split"][~data_split_pd[0].isna()] = "test"

adata.obs["split"] = data_split_pd.split.values.tolist()

# create adata test object
adata_test = adata[adata.obs.split == "test"]

#### add image path to adata object

In [5]:
adata_test.obs["img_path"] = test_image_path + "/" + adata_test.obs.obs_cls + "/" + adata_test.obs.obs_id

Trying to set attribute `.obs` of view, copying.


### Preprocessing

In [ ]:
# outlier filtering, remove any images with less 2 then 1000

In [9]:
sc.pl.highest_expr_genes(adata_test, log=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
sc.pl.violin(adata_test, var_names, log=True)

... storing 'obs_id' as categorical
... storing 'obs_cls' as categorical
... storing 'split' as categorical
... storing 'img_path' as categorical


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/olle/miniconda3/envs/visualization/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/home/olle/miniconda3/envs/visualization/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


#### filter_data

In [11]:
adata = adata[adata.X[:, 2] > 1000]

In [ ]:
# Log and norm data

In [12]:
sc.pp.log1p(adata_test)

In [13]:
sc.pp.highly_variable_genes(adata_test, min_mean=0.000125, max_mean=10000, min_disp=0.5)
sc.pl.highly_variable_genes(adata_test)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
sc.pp.scale(adata_test, max_value=10)


In [ ]:
### PCA analysis

In [15]:
sc.tl.pca(adata_test,n_comps=10, svd_solver='arpack')


In [16]:
sc.pl.pca(adata_test, color='obs_cls')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
sc.pl.pca_variance_ratio(adata_test, log=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
##### compte neighbour hood graph

In [18]:
sc.pp.neighbors(adata_test, n_neighbors=100, n_pcs=40)
sc.tl.umap(adata_test)

/home/olle/miniconda3/envs/visualization/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9002. The TBB threading layer is disabled.
  warnings.warn(problem)


In [19]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
sc.pl.umap(adata_test, color=['obs_cls', "1", "3", "4", "7", "5", "8", "9", "10", "13"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.cbook import get_sample_data
import cv2
import matplotlib.patches as mpatches
%matplotlib widget


def main(data):
    x = data.obsm["X_umap"][:, 0]
    y = data.obsm["X_umap"][:, 1]

    color_map = {'NORMAL': 'tab:blue', 'DRUSEN': 'tab:green', "CNV": 'tab:orange', 'DME': 'tab:red'}

    colors = [color_map[class_] for class_ in adata_test.obs.obs_cls.values.tolist()]
    
    fig, ax = plt.subplots(figsize=(20,10))
    #imscatter(x, y, data.obs.img_path.values.tolist(), zoom=0.05, ax=ax)
    ax.scatter(x, y, c=colors, alpha=0.5)
    plt.legend(np.unique(color_map.keys()))
    
    patches = []
    for key in color_map.keys():
        color = color_map[key]
        patches.append(mpatches.Patch(color=color, label=key))
    
    plt.legend(handles=patches)
    #plt.savefig("test.png")
    plt.show()

def imscatter(x, y, image_path, ax=None, zoom=2):
    if ax is None:
        ax = plt.gca()
    x, y = np.atleast_1d(x, y)
    artists = []
    iter_ = 0
    for x0, y0 in zip(x, y):
        if iter_ % 10 == 0:
            image = plt.imread(image_path[iter_])
            image = np.stack((image,) * 3, axis = -1)
            im = OffsetImage(image, zoom=zoom)
            ab = AnnotationBbox(im, (x0, y0), xycoords='data', frameon=False)
            artists.append(ax.add_artist(ab))
            
        iter_ += 1
    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()
    return artists

main(adata_test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
adata_test.obs['img_path'].iloc[0]

'/home/olle/PycharmProjects/LODE/workspace/feature_statistics/cell_data/OCT2017/test/NORMAL/NORMAL-98720-1.jpeg'

In [87]:
X = adata_test.X


color_map = {'NORMAL': 0, 'DRUSEN': 1, "CNV": 2, 'DME': 3}

y = [color_map[class_] for class_ in adata_test.obs.obs_cls.values.tolist()]

groups = list(enumerate(adata_test.obs.obs_id.values.tolist()))

In [88]:
groups = [obj[0] for obj in groups]

In [112]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data
    ('clf', LogisticRegression()) #step2 - classifier
])
pipeline.steps


adata_test.obs['obs_cls'] = LabelEncoder().fit_transform(adata_test.obs['obs_cls'])

#Seperate train and test data
X_train, X_test, y_train, y_test = train_test_split(adata_test.X[:,:-1],
                                                   adata_test.obs.obs_cls,
                                                   test_size = 0.4,
                                                   random_state = 10)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(802, 15)
(535, 15)
(802,)
(535,)


In [114]:
from sklearn.model_selection import cross_validate

scores = cross_validate(pipeline, X_train, y_train)
scores['test_score'].mean()


0.8727950310559006

In [ ]:
#### Clustering

In [115]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clfs = []
clfs.append(LogisticRegression())
clfs.append(SVC())
clfs.append(SVC())
clfs.append(KNeighborsClassifier(n_neighbors=3))
clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(GradientBoostingClassifier())

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X_train, y_train)
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            print(key,' std ', values.std())

---------------------------------
LogisticRegression()
-----------------------------------
fit_time  mean  0.03793015480041504
fit_time  std  0.005283121955516961
score_time  mean  0.0006619453430175781
score_time  std  1.849157039172333e-05
test_score  mean  0.8727950310559006
test_score  std  0.011137486631380514
---------------------------------
SVC()
-----------------------------------
fit_time  mean  0.017557239532470702
fit_time  std  0.000486193361620677
score_time  mean  0.0030720710754394533
score_time  std  4.226870265902778e-05
test_score  mean  0.8765295031055901
test_score  std  0.012966573044243751
---------------------------------
SVC()
-----------------------------------
fit_time  mean  0.018161773681640625
fit_time  std  0.0020919708321574435
score_time  mean  0.003406333923339844
score_time  std  0.0006683729067899503
test_score  mean  0.8765295031055901
test_score  std  0.012966573044243751
---------------------------------
KNeighborsClassifier(n_neighbors=3)
-------

In [ ]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1)

clf.fit(X, y)
print(clf.score(X, y))
print(clf.best_params_)

In [118]:
result = clf.predict(testX)

{'clf__C': 0.7, 'clf__kernel': 'rbf'}

In [119]:
cv_grid.best_estimator_

Pipeline(steps=[('normalizer', StandardScaler()), ('clf', SVC(C=0.7))])

In [120]:
cv_grid.best_score_

0.8827406832298136

In [121]:
y_predict = cv_grid.predict(X_test)
accuracy = accuracy_score(y_test,y_predict)
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))

Accuracy of the best classifier after CV is 88.037%


In [ ]:
#### Feature expression significance

In [ ]:
sc.tl.rank_genes_groups(adata, 'obs_cls', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.pl.violin(adata, ["3", "5", "13"], groupby='obs_cls')


In [ ]:
sc.pl.violin(adata, [ "7", "8", "9"], groupby='obs_cls')

In [ ]:
#### Self labeling

In [ ]:
confusion_matrix = np.zeros((num_classes, num_classes))

# For each class.
for class_idx in range(num_classes):
    # Consider 10 examples.
    example_idxs = class_idx_to_test_idxs[class_idx][:10]
    for y_test_idx in example_idxs:
        # And count the classes of its near neighbours.
        for nn_idx in near_neighbours[y_test_idx][:-1]:
            nn_class_idx = y_test[nn_idx]
            confusion_matrix[class_idx, nn_class_idx] += 1

# Display a confusion matrix.
labels = [
    "Airplane",
    "Automobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=labels)
disp.plot(include_values=True, cmap="viridis", ax=None, xticks_rotation="vertical")
plt.show()

In [ ]:
sc.pl.dotplot(adata, adata.var_names, groupby='obs_cls');

In [ ]:
sc.pl.dotplot(adata, adata.var_names, groupby='leiden');

In [ ]:
# normalize
X_norm = sc.pp.normalize_total(adata, target_sum=1, inplace=False)['X']

In [ ]:
X_norm.shape